In [27]:
# Import Libraries and Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
from fuzzywuzzy import process
import pandas as pd
import requests
import time

# World Cities DB Cleaning

In [28]:
# Read csv and create DF
worldcities_df = pd.read_csv("./RESOURCES/worldcities.csv")
worldcities_df.drop(columns=worldcities_df.columns[0], inplace=True)

worldcities_df

,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37732000.0,1392685764
1,Jakarta,-6.1750,106.8275,Indonesia,ID,IDN,Jakarta,primary,33756000.0,1360771077
2,Delhi,28.6100,77.2300,India,IN,IND,Delhi,admin,32226000.0,1356872604
3,Guangzhou,23.1300,113.2600,China,CN,CHN,Guangdong,admin,26940000.0,1156237133
4,Mumbai,19.0761,72.8775,India,IN,IND,Mahārāshtra,admin,24973000.0,1356226629
...,...,...,...,...,...,...,...,...,...,...
47863,Munha-dong,39.3813,127.2517,"Korea, North",KP,PRK,Kangwŏn,NaN,NaN,1408979215
47864,Sil-li,39.4880,125.4640,"Korea, North",KP,PRK,P’yŏngnam,NaN,NaN,1408767958
47865,Muan,34.9897,126.4714,"Korea, South",KR,KOR,Jeonnam,admin,NaN,1410001061
47866,Hongseong,36.6009,126.6650,"Korea, South",KR,KOR,Chungnam,admin,NaN,1410822139


In [29]:
# Adjust the column name as necessary based on your dataset
filtered_df = worldcities_df[worldcities_df['country'] == 'Mexico'].copy()

# Display the first few rows of the filtered data
print(filtered_df.head())

      city_ascii      lat       lng country iso2 iso3        admin_name  \
9    Mexico City  19.4333  -99.1333  Mexico   MX  MEX  Ciudad de México   
109  Guadalajara  20.6767 -103.3475  Mexico   MX  MEX           Jalisco   
115    Monterrey  25.6667 -100.3000  Mexico   MX  MEX        Nuevo León   
416      Tijuana  32.5250 -117.0333  Mexico   MX  MEX   Baja California   
427     Ecatepec  19.6097  -99.0600  Mexico   MX  MEX            México   

     capital  population          id  
9    primary  21804000.0  1484247881  
109    admin   5525000.0  1484950208  
115    admin   5341171.0  1484559591  
416    minor   2002000.0  1484708778  
427    minor   1929926.0  1484003694  


In [30]:
# Check how many rows there are in the filtered worldcities_df
num_rows = filtered_df.shape[0]
print(f"Number of rows in the filtered DataFrame: {num_rows}")

Number of rows in the filtered DataFrame: 983


In [31]:
filtered_df.columns

Index(['city_ascii', 'lat', 'lng', 'country', 'iso2', 'iso3', 'admin_name',
       'capital', 'population', 'id'],
      dtype='object')

In [32]:
filtered_df.drop(columns=['country', 'iso2', 'iso3', 'capital', 'population'], inplace=True)
filtered_df.columns

Index(['city_ascii', 'lat', 'lng', 'admin_name', 'id'], dtype='object')

In [33]:
filtered_df

,city_ascii,lat,lng,admin_name,id
9,Mexico City,19.4333,-99.1333,Ciudad de México,1484247881
109,Guadalajara,20.6767,-103.3475,Jalisco,1484950208
115,Monterrey,25.6667,-100.3000,Nuevo León,1484559591
416,Tijuana,32.5250,-117.0333,Baja California,1484708778
427,Ecatepec,19.6097,-99.0600,México,1484003694
...,...,...,...,...,...
46873,Esperanza,27.5800,-109.9298,Sonora,1484729183
47008,Nautla,20.2167,-96.7833,Veracruz,1484174535
47143,Altata,24.6333,-107.9167,Sinaloa,1484699069
47305,Mazatan,29.0167,-110.1333,Sonora,1484350756


In [35]:
# Rename the columns
filtered_df.rename(columns={
    'city_ascii': 'City',
    'admin_name': 'State',
    'lat': 'Lat',
    'lng': 'Lng',
    'id': 'ID'
}, inplace=True)

# Reorder the columns
filtered_df = filtered_df[['City', 'State', 'Lat', 'Lng', 'ID']]

filtered_df

,City,State,Lat,Lng,ID
9,Mexico City,Ciudad de México,19.4333,-99.1333,1484247881
109,Guadalajara,Jalisco,20.6767,-103.3475,1484950208
115,Monterrey,Nuevo León,25.6667,-100.3000,1484559591
416,Tijuana,Baja California,32.5250,-117.0333,1484708778
427,Ecatepec,México,19.6097,-99.0600,1484003694
...,...,...,...,...,...
46873,Esperanza,Sonora,27.5800,-109.9298,1484729183
47008,Nautla,Veracruz,20.2167,-96.7833,1484174535
47143,Altata,Sinaloa,24.6333,-107.9167,1484699069
47305,Mazatan,Sonora,29.0167,-110.1333,1484350756


In [36]:
# Save the modified DataFrame to a new CSV file
filtered_df.to_csv('./RESOURCES/mexico_cities.csv', index=False)

CON CSV

In [100]:
# Read csv and create DF

UFO_df = pd.read_csv("./RESOURCES/nuforc_mexico_reports_detailed.csv")

UFO_df.drop(columns=UFO_df.columns[0], inplace=True)

UFO_df

,Occurred,City,State,Country,Shape,Summary,Reported,Media,Explanation,Detail URL,Detailed Description
0,05/08/1998 22:30,Morelos (rural area) (Mexico),NaN,Mexico,Light,We where on the road when we saw 2 long lights...,07/27/1998,NaN,NaN,https://nuforc.org/sighting/?id=4301,NaN
1,03/15/1996 03:00,San Felipe (driving south to) (Mexico),NaN,Mexico,Diamond,While traveling to San Felipe for a fishing tr...,03/12/1998,NaN,NaN,https://nuforc.org/sighting/?id=3323,NaN
2,09/15/1998 21:00,"Wadley, San Luis Potosi, SLP (In the desert ne...",NaN,Mexico,Disk,"Craft was approaching, it accelerated very fas...",11/10/1998,NaN,NaN,https://nuforc.org/sighting/?id=4800,NaN
3,09/15/1994 10:00,Mexicali B. C. Mexico; Sierra Cucapah; Cerro p...,NaN,Mexico,Oval,I seen one objet upper the Sierra cucapah (Cuc...,01/03/1998,NaN,NaN,https://nuforc.org/sighting/?id=3007,NaN
4,12/05/1998 19:00,Valle de Bravo (Mexico),NaN,Mexico,Light,White bright light like a headlight with a clo...,12/06/1998,NaN,NaN,https://nuforc.org/sighting/?id=5016,NaN
...,...,...,...,...,...,...,...,...,...,...,...
313,04/27/2000 21:00,"San Felipe, Baja de California (Mexico)",NaN,Mexico,NaN,Something in the night sky!,05/07/2000,NaN,NaN,https://nuforc.org/sighting/?id=12815,NaN
314,06/15/1999 13:00,Chihuahua (Mexico),NaN,Mexico,Other,"I think I saw contrails, but this day some mil...",11/03/1999,NaN,NaN,https://nuforc.org/sighting/?id=10416,NaN
315,06/01/1994 21:30,Ensenada (Baja Calif.) (Mexico),NaN,Mexico,Light,"while scanning the stars,I was drawn to a perf...",02/03/2000,NaN,NaN,https://nuforc.org/sighting/?id=11872,NaN
316,06/14/1999 22:30,San Felipe (Baja) (Mexico) (Mexico),NaN,Mexico,Light,Two lights darting in a zig zag patern. Anothe...,08/05/1999,NaN,NaN,https://nuforc.org/sighting/?id=8578,NaN


In [101]:
# Delete spaces on headers (required for the next step, since there are spaces before/after titles)
# For example [' Explanation '] to ['Explanation']
UFO_df.rename(columns=lambda x: x.strip(), inplace=True)

# Drop columns that don't give relevant information
UFO_df.drop(columns=['Explanation','Reported','Detailed Description','Media'], inplace=True)

In [102]:
print(UFO_df.columns)

Index(['Occurred', 'City', 'State', 'Country', 'Shape', 'Summary',
       'Detail URL'],
      dtype='object')


In [103]:
# Create a new column with the information that is in parenthesis in City

UFO_df['Notas'] = UFO_df['City'].str.extract(r'\((.*?)\)')

# Delete the information in parenthesis in the column City

UFO_df['City']=UFO_df['City'].str.replace(r'\((.*?)\)','',regex=True)

UFO_df

,Occurred,City,State,Country,Shape,Summary,Detail URL,Notas
0,05/08/1998 22:30,Morelos,NaN,Mexico,Light,We where on the road when we saw 2 long lights...,https://nuforc.org/sighting/?id=4301,rural area
1,03/15/1996 03:00,San Felipe,NaN,Mexico,Diamond,While traveling to San Felipe for a fishing tr...,https://nuforc.org/sighting/?id=3323,driving south to
2,09/15/1998 21:00,"Wadley, San Luis Potosi, SLP",NaN,Mexico,Disk,"Craft was approaching, it accelerated very fas...",https://nuforc.org/sighting/?id=4800,In the desert near
3,09/15/1994 10:00,Mexicali B. C. Mexico; Sierra Cucapah; Cerro p...,NaN,Mexico,Oval,I seen one objet upper the Sierra cucapah (Cuc...,https://nuforc.org/sighting/?id=3007,NaN
4,12/05/1998 19:00,Valle de Bravo,NaN,Mexico,Light,White bright light like a headlight with a clo...,https://nuforc.org/sighting/?id=5016,Mexico
...,...,...,...,...,...,...,...,...
313,04/27/2000 21:00,"San Felipe, Baja de California",NaN,Mexico,NaN,Something in the night sky!,https://nuforc.org/sighting/?id=12815,Mexico
314,06/15/1999 13:00,Chihuahua,NaN,Mexico,Other,"I think I saw contrails, but this day some mil...",https://nuforc.org/sighting/?id=10416,Mexico
315,06/01/1994 21:30,Ensenada,NaN,Mexico,Light,"while scanning the stars,I was drawn to a perf...",https://nuforc.org/sighting/?id=11872,Baja Calif.
316,06/14/1999 22:30,San Felipe,NaN,Mexico,Light,Two lights darting in a zig zag patern. Anothe...,https://nuforc.org/sighting/?id=8578,Baja


In [104]:
# Check the cities that are contained the most to update the state

UFO_df['City'].value_counts()

City
Cancun                  25
Puerto Vallarta         17
Mexico City             16
Tijuana                 15
Mexico                  12
                        ..
Playa Del Carmen         1
Mexico City              1
Apodaca, Nuevo Leon      1
Pescadero                1
Teotihuacan              1
Name: count, Length: 169, dtype: int64

In [107]:
repeated_cities = {
    "Cancun": "Quintana Roo",
    "Puerto Vallarta": "Jalisco",
    "Mexico City": "Mexico City",
    "Tijuana": "Baja California"
}

# Assign state based on city
def assing_city(row):
    for city, state in repeated_cities.items():
        if pd.notna(row['City']) and city.lower() in row['City'].lower():
            return state
    return row['State']

# Apply the function
UFO_df['State'] = UFO_df.apply(assing_city, axis=1)


In [108]:
# Set a list of the name of the Mexican States
state_names = ['Aguascalientes', 'Baja California', 'Baja California Sur', 'Campeche', 'Chiapas', 'Chihuahua', 'Coahuila', 'Colima', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo', 'Jalisco', 'Estado de Mexico', 'Michoacan', 'Morelos', 'Nayarit', 'Nuevo Leon', 'Oaxaca', 'Puebla', 'Queretaro', 'Quintana Roo', 'San Luis Potosi', 'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas', 'Tlaxcala', 'Veracruz', 'Yucatan', 'Zacatecas','Mexico City']

#define a function that loops on the city name to find the state
def update_state(row):
    for state in state_names:
        if pd.notna(row['City']) and state.lower() in row['City'].lower():
            return state
    return row['State']

# Apply the function to the State column
UFO_df['State'] = UFO_df.apply(update_state, axis=1)

UFO_df

,Occurred,City,State,Country,Shape,Summary,Detail URL,Notas
0,05/08/1998 22:30,Morelos,Morelos,Mexico,Light,We where on the road when we saw 2 long lights...,https://nuforc.org/sighting/?id=4301,rural area
1,03/15/1996 03:00,San Felipe,NaN,Mexico,Diamond,While traveling to San Felipe for a fishing tr...,https://nuforc.org/sighting/?id=3323,driving south to
2,09/15/1998 21:00,"Wadley, San Luis Potosi, SLP",San Luis Potosi,Mexico,Disk,"Craft was approaching, it accelerated very fas...",https://nuforc.org/sighting/?id=4800,In the desert near
3,09/15/1994 10:00,Mexicali B. C. Mexico; Sierra Cucapah; Cerro p...,NaN,Mexico,Oval,I seen one objet upper the Sierra cucapah (Cuc...,https://nuforc.org/sighting/?id=3007,NaN
4,12/05/1998 19:00,Valle de Bravo,NaN,Mexico,Light,White bright light like a headlight with a clo...,https://nuforc.org/sighting/?id=5016,Mexico
...,...,...,...,...,...,...,...,...
313,04/27/2000 21:00,"San Felipe, Baja de California",NaN,Mexico,NaN,Something in the night sky!,https://nuforc.org/sighting/?id=12815,Mexico
314,06/15/1999 13:00,Chihuahua,Chihuahua,Mexico,Other,"I think I saw contrails, but this day some mil...",https://nuforc.org/sighting/?id=10416,Mexico
315,06/01/1994 21:30,Ensenada,NaN,Mexico,Light,"while scanning the stars,I was drawn to a perf...",https://nuforc.org/sighting/?id=11872,Baja Calif.
316,06/14/1999 22:30,San Felipe,NaN,Mexico,Light,Two lights darting in a zig zag patern. Anothe...,https://nuforc.org/sighting/?id=8578,Baja


In [116]:
# Check if the state is in the Notas column if the State is still not contained in the column

def update_withnote(row):
    if pd.isna(row['State']): 
        for state in state_names:
            if pd.notna(row['Notas']) and state.lower() in row['Notas'].lower():
                return state
    return row['State'] 

UFO_df['State'] = UFO_df.apply(update_withnote, axis=1)

UFO_df

,Occurred,City,State,Country,Shape,Summary,Detail URL,Notas
0,05/08/1998 22:30,Morelos,Morelos,Mexico,Light,We where on the road when we saw 2 long lights...,https://nuforc.org/sighting/?id=4301,rural area
1,03/15/1996 03:00,San Felipe,NaN,Mexico,Diamond,While traveling to San Felipe for a fishing tr...,https://nuforc.org/sighting/?id=3323,driving south to
2,09/15/1998 21:00,"Wadley, San Luis Potosi, SLP",San Luis Potosi,Mexico,Disk,"Craft was approaching, it accelerated very fas...",https://nuforc.org/sighting/?id=4800,In the desert near
3,09/15/1994 10:00,Mexicali B. C. Mexico; Sierra Cucapah; Cerro p...,NaN,Mexico,Oval,I seen one objet upper the Sierra cucapah (Cuc...,https://nuforc.org/sighting/?id=3007,NaN
4,12/05/1998 19:00,Valle de Bravo,NaN,Mexico,Light,White bright light like a headlight with a clo...,https://nuforc.org/sighting/?id=5016,Mexico
...,...,...,...,...,...,...,...,...
313,04/27/2000 21:00,"San Felipe, Baja de California",Baja California,Mexico,NaN,Something in the night sky!,https://nuforc.org/sighting/?id=12815,Mexico
314,06/15/1999 13:00,Chihuahua,Chihuahua,Mexico,Other,"I think I saw contrails, but this day some mil...",https://nuforc.org/sighting/?id=10416,Mexico
315,06/01/1994 21:30,Ensenada,NaN,Mexico,Light,"while scanning the stars,I was drawn to a perf...",https://nuforc.org/sighting/?id=11872,Baja Calif.
316,06/14/1999 22:30,San Felipe,NaN,Mexico,Light,Two lights darting in a zig zag patern. Anothe...,https://nuforc.org/sighting/?id=8578,Baja


SCRAPPING

In [2]:
# Set up Splinter
browser = Browser('chrome')

In [3]:
# Visit the NUFORC REPORTS UFO SIGHTINGS site
url = 'https://nuforc.org/subndx/?id=cMexico'
browser.visit(url)

In [4]:
# Scrape the website
html = browser.html

# Create a BeautifulSoup object from the scraped HTML
soup = BeautifulSoup(html, 'html.parser')

In [5]:
# Initialize variables to store scraped data
all_data = []  # List to store table data
links = []     # List to store links
# Loop until the "Next" link is clicked 4 times
# The underscore means "iterate four times, and I don't care about the loop variable."
for _ in range(4):
    
    ''' EXTRACT DATA AND LINKS FROM THE TABLE INTO LISTS
     '#table_1 tr' is a CSS selector used to select all <tr> (table row) elements within
     the HTML table that has an id attribute equal to 'table_1'.'''
    
    table_rows = soup.select('#table_1 tr')[1:]  # # the [1:] means skip the first row (headers)
    for row in table_rows:
        # Extract text data from each <td> element in the row and removes leading and trailing whitespace characters from each extracted text
        row_data = [td.get_text(strip=True) for td in row.find_all('td')]
        all_data.append(row_data)  # Append row data to the list of all data
        # Extract links from <a> elements in the row
        row_links = [f"https://nuforc.org{a['href']}" for a in row.find_all('a', href=True)]
        links.extend(row_links)  # Extend the list of links with links from the current row
    # Find the "Next" link and click it
    next_link = browser.find_by_id('table_1_next')
    next_link.click()
    # Wait for the page to load
    time.sleep(2)  # Adjust the sleep time as needed
    # Update the HTML content after navigating to the next page
    html = browser.html
    # Create a new BeautifulSoup object from the updated HTML
    soup = BeautifulSoup(html, 'html.parser')
# Extract headers from the table
headers = [header.get_text(strip=True) for header in soup.select('#table_1 th')]
# Convert the list of lists into a pandas DataFrame
table_df = pd.DataFrame(all_data, columns=headers)  # Create DataFrame for table data
links_df = pd.DataFrame(links, columns=['Link'])    # Create DataFrame for links


     Link          Occurred                City                State Country  \
0    Open  05/15/2024 02:20            Mazatlán              Sinaloa  Mexico   
1  Open !  05/12/2024 20:58         Los Médanos      Baja California  Mexico   
2    Open  05/09/2024 21:38   San José del Cabo  Baja California Sur  Mexico   
3    Open  02/26/2024 19:40    Ciudad de México     Ciudad de México  Mexico   
4    Open  12/28/2023 20:49  Acapulco de Juárez             Guerrero  Mexico   

      Shape                                            Summary Media  \
0     Light  Lights dropping from very high altitude and th...         
1  Cylinder  We observed a very large (500-1500 ft long), d...         
2  Triangle  We saw a triangular/round bright light in the ...     Y   
3       Orb  I was walking down the street coming to my hom...     Y   
4    Circle                           Maybe drone or maybe ufo     Y   

  Explanation  
0              
1              
2      Rocket  
3              
4     

In [6]:
    # Display the table_df
table_df

,Link,Occurred,City,State,Country,Shape,Summary,Media,Explanation
0,Open,05/15/2024 02:20,Mazatlán,Sinaloa,Mexico,Light,Lights dropping from very high altitude and th...,,
1,Open !,05/12/2024 20:58,Los Médanos,Baja California,Mexico,Cylinder,"We observed a very large (500-1500 ft long), d...",,
2,Open,05/09/2024 21:38,San José del Cabo,Baja California Sur,Mexico,Triangle,We saw a triangular/round bright light in the ...,Y,Rocket
3,Open,02/26/2024 19:40,Ciudad de México,Ciudad de México,Mexico,Orb,I was walking down the street coming to my hom...,Y,
4,Open,12/28/2023 20:49,Acapulco de Juárez,Guerrero,Mexico,Circle,Maybe drone or maybe ufo,Y,Drone?
...,...,...,...,...,...,...,...,...,...
399,Open,02/16/1984 23:00,"Guadalajara, Jalisco (Mexico)",,Mexico,Formation,it happend while on a road to guadalajara jali...,,
400,Open,06/01/1983 20:30,Ciudad Lazaro Cardenas (Mexico),,Mexico,Sphere,We saw a gigantic sphere in the sky one Saturd...,,
401,Open,05/03/1983 22:00,Highway (Mexico),,Mexico,Oval,Huge light formation spanned across (and above...,,
402,Open,08/01/1982 13:00,Cozumel (Mexico),,Mexico,,Can someone give me an idea what this is?,,


In [7]:
# Display the links_df
links_df.head()

,Link
0,https://nuforc.org/sighting/?id=181515
1,https://nuforc.org/sighting/?id=181495
2,https://nuforc.org/sighting/?id=181446
3,https://nuforc.org/sighting/?id=180712
4,https://nuforc.org/sighting/?id=179884


In [8]:
# Drop the first column in the table_df
table_df.drop(columns=table_df.columns[0], inplace=True)

# Merge both dataframes to include the links column (replacing the dropped one)
merged_df = pd.merge(table_df, links_df, left_index=True, right_index=True)
merged_df.head()

,Occurred,City,State,Country,Shape,Summary,Media,Explanation,Link
0,05/15/2024 02:20,Mazatlán,Sinaloa,Mexico,Light,Lights dropping from very high altitude and th...,,,https://nuforc.org/sighting/?id=181515
1,05/12/2024 20:58,Los Médanos,Baja California,Mexico,Cylinder,"We observed a very large (500-1500 ft long), d...",,,https://nuforc.org/sighting/?id=181495
2,05/09/2024 21:38,San José del Cabo,Baja California Sur,Mexico,Triangle,We saw a triangular/round bright light in the ...,Y,Rocket,https://nuforc.org/sighting/?id=181446
3,02/26/2024 19:40,Ciudad de México,Ciudad de México,Mexico,Orb,I was walking down the street coming to my hom...,Y,,https://nuforc.org/sighting/?id=180712
4,12/28/2023 20:49,Acapulco de Juárez,Guerrero,Mexico,Circle,Maybe drone or maybe ufo,Y,Drone?,https://nuforc.org/sighting/?id=179884


In [ ]:
base_url = "https://nuforc.org/subndx/?id=cMexico&page="
# Número de páginas a scraper
num_pages = 5  # Ajusta este valor según sea necesario
# Crear un DataFrame para almacenar los datos
df = pd.DataFrame()
# Iterar sobre las páginas
for page in range(1, num_pages + 1):
    # Construir la URL para la página actual
    url = base_url + str(page)
    # Enviar solicitud GET para la página actual
    response = requests.get(url)
    # Parsear el contenido HTML con BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    # Encontrar la tabla que contiene los datos
    table = soup.find('table', {'class': 'table-striped'})
    # Extraer los encabezados de la tabla
    headers = [th.text.strip() for th in table.find('tr').find_all('th')]
    # Extraer los datos de la tabla
    data = []
    for row in table.find_all('tr')[1:]:
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols]
        data.append([col for col in cols])
    # Convertir los datos en un DataFrame de Pandas
    page_df = pd.DataFrame(data, columns=headers)
    # Agregar los datos de la página actual al DataFrame principal
    df = pd.concat([df, page_df], ignore_index=True)
# Mostrar el DataFrame completo
print(df.head())